In [1]:
!pip install phe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 723.7 kB/s eta 0:00:00


In [2]:
import random
import requests
import pickle
from phe import paillier

In [3]:
# Get the public key
response = requests.get('https://helaly6auc.pythonanywhere.com/get_public_key')
filename = "pickled_public_key.pb"
with open(filename,'wb') as f:
  f.write(response.content)

In [4]:
import numpy as np

In [5]:
# Retreive Public key from auth server public and private key pair
with open('/content/pickled_public_key.pb','rb') as file:
  pickled_public_key = file.read()

unpickled_public_key = pickle.loads(pickled_public_key)
# Assume that the number of minions is 2
class user:
  def __init__(self,personalData,public_key):
    self._personal_val = personalData;
    self.public_key= unpickled_public_key
    self._personal_data_split = [];
    self._personal_data_split_encrypted = [];
    self.divide_val_to_shares(2)
    self.encrypt_shares()
    self.binaries_of_votes= self.binary_representation()
    self.binaries_of_votes_encrypted=[]
    self.encrypt_binaries()
    self.url1= "https://yomna6auc.pythonanywhere.com/send_shares_minion1"
    self.url2= "https://yomna6auc.pythonanywhere.com/send_shares_minion2"
    self.url3= "https://yomna6auc.pythonanywhere.com/send_binary_answer"


  def encrypt_shares(self):
    for plain_answer_share_vector in self._personal_data_split:
      encryptedAnswer = [ self.public_key.encrypt(plain_text_share_vector) for plain_text_share_vector in plain_answer_share_vector]
      self._personal_data_split_encrypted.append(encryptedAnswer)

  def divide_val_to_shares(self, n):
    shares = []
    answerNo = 1
    for v in self._personal_val:
      answerNo=answerNo+1
      shares = [random.randint(-10,10) for _ in range(n-1)]
      shares.append(v - sum(shares))
      self._personal_data_split.append(shares)

  def binary_representation(self):
    binary = []
    for i in self._personal_val:
      binary_t = [0]*5
      if 1 <= i <= 5:
        binary_t[i - 1] = 1
        binary.append(binary_t)
    return binary

  def convert_to_numpy(self):
    self._personal_data_split_encrypted=np.array(self._personal_data_split_encrypted)
    self._personal_data_split=np.array(self._personal_data_split)

  def encrypt_binaries(self):
      for plain_answer_share_vector in self.binaries_of_votes:
        encryptedAnswer = [ self.public_key.encrypt(plain_text_share_vector) for plain_text_share_vector in plain_answer_share_vector]
        self.binaries_of_votes_encrypted.append(encryptedAnswer)

  def pickle_and_send(self,data_to_send,url):
      # convert the encrypted result to pickles
      pickled_result_from_poll_server = pickle.dumps(data_to_send,protocol=2)
      # Header to be used  in the post request
      headers = {'Content-Type': 'application/octet-stream'}
      # send the encrypted data to the remote auth server to decrypt it
      try:
          response = requests.post(url, data=pickled_result_from_poll_server, headers=headers)
          response.raise_for_status()
          print(response.text)
      except requests.exceptions.RequestException as e:
          print(f"Error: {e}")
          print(response.text)

  def send_to_minion_1(self):
    data_to_send= self._personal_data_split_encrypted[:,0].tolist()
    self.pickle_and_send(data_to_send,self.url1)

  def send_to_minion_2(self):
    data_to_send= self._personal_data_split_encrypted[:,1].tolist()
    self.pickle_and_send(data_to_send,self.url2)


  def send_to_minion_binary(self):
    data_to_send= self.binaries_of_votes_encrypted
    self.pickle_and_send(data_to_send,self.url3)


In [6]:
U1= user([4,3,3,1],unpickled_public_key)
U1.encrypt_shares()
U1.convert_to_numpy()
U1.send_to_minion_1()
U1.send_to_minion_2()
U1.send_to_minion_binary()
#######################
U2= user([2,3,1,5],unpickled_public_key)
U2.encrypt_shares()
U2.convert_to_numpy()
U2.send_to_minion_1()
U2.send_to_minion_2()
U2.send_to_minion_binary()
#######################
U3= user([2,1,3,4],unpickled_public_key)
U3.encrypt_shares()
U3.convert_to_numpy()
U3.send_to_minion_1()
U3.send_to_minion_2()
U3.send_to_minion_binary()
#######################
response = requests.get('https://yomna6auc.pythonanywhere.com/run_stats')
print(response.text)
#######################
response = requests.get('https://yomna6auc.pythonanywhere.com/run_stats_mod')
print(response.text)

{"data shares in minion 1: ":"hello minion 1"}

{"data shares in minion 1: ":"hello minion 1"}

{"data shares in minion 2: ":"hello minion 2"}

{"data shares in minion 1: ":"hello minion 1"}

{"data shares in minion 1: ":"hello minion 1"}

{"data shares in minion 2: ":"hello minion 2"}

{"data shares in minion 1: ":"hello minion 1"}

{"data shares in minion 1: ":"hello minion 1"}

{"data shares in minion 2: ":"hello minion 2"}

{"success":"success in receiving"}

{"success":"success in receiving binary matrix"}



In [ ]:
# response = requests.get('https://yomna6auc.pythonanywhere.com/run_stats')
# print(response.text)

In [ ]:
# response = requests.get('https://yomna6auc.pythonanywhere.com/run_stats_mod')
# print(response.text)

{"success":"success in receiving binary matrix"}

